In [1]:
import os

In [ ]:
%pwd

'c:\\Users\\Ganesh\\AIML Engineer\\DL (End-to-End Project)\\End-to-End-Chest-Cancer-Classification-using-MLFlow-DVC-Deep-Learning-Project-\\research'

In [ ]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Ganesh\\AIML Engineer\\DL (End-to-End Project)\\End-to-End-Chest-Cancer-Classification-using-MLFlow-DVC-Deep-Learning-Project-'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/DataWithGanesh/End-to-End-Chest-Cancer-Classification-using-MLFlow-DVC-Deep-Learning-Project-.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "DataWithGanesh"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "ab2ede637e99a9bf83b9da9e605ec45255bc088f"

In [6]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
from pathlib import Path
from cnnClassifier.entity.config_entity import EvaluationConfig
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            mlflow_uri="https://dagshub.com/entbappy/chest-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            experiment_name="chest_cancer_classification"
        )
        return eval_config


In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

c:\Users\Ganesh\anaconda3\envs\cancer\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [23]:
import tensorflow as tf
from pathlib import Path
from cnnClassifier.utils.common import save_json

class Evaluation:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.3
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        import mlflow
        import mlflow.tensorflow

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment(self.config.experiment_name)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.tensorflow.log_model(
                tf_model=self.model,
                artifact_path="model",
                registered_model_name="VGG16Model"
            )


In [24]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-12-04 12:45:30,474: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-04 12:45:30,479: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-04 12:45:30,482: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 12s 2s/step - loss: 26.8452 - accuracy: 0.5686
[2025-12-04 12:45:43,176: INFO: common: json file saved at: scores.json]


MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 404 != 200. Response body: ''